In [1]:
from __future__ import print_function
import sys, os, math
import h5py
import numpy as np
from numpy import float32, int32, uint8, dtype

# Load PyGreentea
# Relative path to where PyGreentea resides
pygt_path = '../../PyGreentea'
sys.path.append(pygt_path)
import PyGreentea as pygt

# Create the network we want
netconf = pygt.netgen.NetConf()
netconf.ignore_conv_buffer = True
netconf.use_batchnorm = False
netconf.dropout = 0.0
netconf.fmap_start = 20
# netconf.unet_fmap_inc_rule = lambda self,fmaps: int(math.ceil(fmaps * 1))
# netconf.unet_fmap_dec_rule = lambda self,fmaps: int(math.ceil(fmaps / 1))

netconf.input_shape = [132,132,132]
netconf.output_shape = [44, 44, 44]

print ('Input shape: %s' % netconf.input_shape)
print ('Output shape: %s' % netconf.output_shape)
print ('Feature maps: %s' % netconf.fmap_start)

netconf.loss_function = "euclid"
train_net_conf_euclid, test_net_conf = pygt.netgen.create_nets(netconf)
netconf.loss_function = "malis"
train_net_conf_malis, test_net_conf = pygt.netgen.create_nets(netconf)

with open('net_train_euclid.prototxt', 'w') as f:
    print(train_net_conf_euclid, file=f)
with open('net_train_malis.prototxt', 'w') as f:
    print(train_net_conf_malis, file=f)
with open('net_test.prototxt', 'w') as f:
    print(test_net_conf, file=f)

Input shape: [132, 132, 132]
Output shape: [44, 44, 44]
Feature maps: 20
f: 1 w: [132, 132, 132] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 20 w: [130, 130, 130] d: [1, 1, 1]
WM: 2160
CM: 248396544
AM: 0
f: 20 w: [128, 128, 128] d: [1, 1, 1]
WM: 43200
CM: 4745520000
AM: 0
f: 20 w: [64, 64, 64] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 60 w: [62, 62, 62] d: [1, 1, 1]
WM: 129600
CM: 566231040
AM: 0
f: 60 w: [60, 60, 60] d: [1, 1, 1]
WM: 388800
CM: 1544365440
AM: 0
f: 60 w: [30, 30, 30] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 180 w: [28, 28, 28] d: [1, 1, 1]
WM: 1166400
CM: 174960000
AM: 0
f: 180 w: [26, 26, 26] d: [1, 1, 1]
WM: 3499200
CM: 426746880
AM: 0
f: 180 w: [13, 13, 13] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 540 w: [11, 11, 11] d: [1, 1, 1]
WM: 10497600
CM: 42709680
AM: 0
f: 540 w: [9, 9, 9] d: [1, 1, 1]
WM: 31492800
CM: 77623920
AM: 0
f: 540 w: [18, 18, 18] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 180 w: [18, 18, 18] d: [1, 1, 1]
WM: 388800
CM: 100776960
AM: 0
f: 360 w: [18, 18, 18] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0

In [4]:
# Biggest possible network for testing on 12 GB
netconf.ignore_conv_buffer = True
netconf.mem_global_limit = 10 * 1024 * 1024 * 1024
mode = pygt.netgen.caffe_pb2.TEST
shape_min = [100,100,100]
shape_max = [500,500,500]
constraints = [None, lambda x: x[0], lambda x: x[1]]

inshape,outshape,fmaps = pygt.netgen.compute_valid_io_shapes(netconf,mode,shape_min,shape_max,constraints=constraints)

# We choose the maximum that still gives us 20 fmaps:
index = [n for n, i in enumerate(fmaps) if i>=20][-1]
print("Index to use: %s" % index)

# Some patching to allow our new parameters
netconf.input_shape = inshape[index]
netconf.output_shape = outshape[index]
# Workaround to allow any size (train net unusably big)
netconf.mem_global_limit = 200 * 1024 * 1024 * 1024
netconf.mem_buf_limit = 200 * 1024 * 1024 * 1024
# Define some loss function (irrelevant for testing though)
netconf.loss_function = "euclid"

# Generate the nework, store it
train_net_big_conf, test_net_big_conf = pygt.netgen.create_nets(netconf)

with open('net_test_big.prototxt', 'w') as f:
    print(test_net_big_conf, file=f)



++++ Valid: [100] => [12]
-- Invalid: [101] => []
-- Invalid: [102] => []
-- Invalid: [103] => []
-- Invalid: [104] => []
-- Invalid: [105] => []
-- Invalid: [106] => []
-- Invalid: [107] => []
++++ Valid: [108] => [20]
-- Invalid: [109] => []
-- Invalid: [110] => []
-- Invalid: [111] => []
-- Invalid: [112] => []
-- Invalid: [113] => []
-- Invalid: [114] => []
-- Invalid: [115] => []
++++ Valid: [116] => [28]
-- Invalid: [117] => []
-- Invalid: [118] => []
-- Invalid: [119] => []
-- Invalid: [120] => []
-- Invalid: [121] => []
-- Invalid: [122] => []
-- Invalid: [123] => []
++++ Valid: [124] => [36]
-- Invalid: [125] => []
-- Invalid: [126] => []
-- Invalid: [127] => []
-- Invalid: [128] => []
-- Invalid: [129] => []
-- Invalid: [130] => []
-- Invalid: [131] => []
++++ Valid: [132] => [44]
-- Invalid: [133] => []
-- Invalid: [134] => []
-- Invalid: [135] => []
-- Invalid: [136] => []
-- Invalid: [137] => []
-- Invalid: [138] => []
-- Invalid: [139] => []
++++ Valid: [140] => [52]
-- I